## Overview
Spring Cloud Gateway is API Gateway implementation by Spring Cloud team on top of Spring reactive ecosystem. Spring Cloud Gateway is built on Spring Boot, Spring WebFlux, and Project Reactor, thereby provides non-blocking asynchronous request processing. Spring Cloud Gateway requires the Netty runtime , it does not work in a traditional Servlet Container or when built as a WAR.  

### Components
- **Route:** comprises of an id, destination URL, collection of predicates and collection of filters.  
- **Predicate:** a predicate function having `ServerWebExchange` as the parameter. `ServerWebExchange` objects binds together the `ServerHttpRequest` and `ServerHttpResponse` objects. A request matches a route based on these predicates.
- **Filter:** instance of `GatewayFilter`, lets you modify request/response before sending downstream.  

Spring Cloud Gateway's *Gateway Handler Mapping* analyses request and determines `Route` associated with it. The request is then sent to *Gateway Web Handler*, which obtains list of `GatewayFilter` objects and passes the request through it.

## Route Configuration
Route can be configured in `application.yml` as well as using Java DSL. Sample configuration:
```yml
spring:
    cloud:
        gateway:
            routes:
                - id: api_route
                  uri: http://api.example.com
                  predicates:
                      - Path=/api
```
The above form is the shortcur form, the fully expanded form looks like:
```yml
spring:
    cloud:
        gateway:
            routes:
                - id: api_route
                  uri: http://api.example.com
                  predicates:
                      - name: Path
                        args:
                            patterns: /api
                            matchTrailingSlash: true
```

### Route Predicates
Spring Cloud bundles a number of predicates that use various components of HTTP request such as path, method, headers, method, etc associate a request with a `Route`. A `RoutePredicateFactory` uses predicate configuration defined in the route definition and creates a `GatewayPredicate` object. This object tests `ServerExchange` object and determines whether it can be associated with a `Route` or not.  

Some useful predefined predicates:
- **Cookie Route Predicate Factory:** takes two parameters, the cookie name and a regexp (which is a Java regular expression). This predicate matches cookies that have the given name and whose values match the regular expression.
```yml
predicates:
    - Cookie=color, g.*
```
Or
```yml
predicates:
    - name: Cookie
      args:
          name: color
          regexp: g.*
```
where color is the name of the cookie and the value is any word starting with g.

- **Header Route Predicate Factory:** very similar to the above
```yml
predicates:
    - Header=X-Request-Id, \d+
```
Or
```yml
predicates:
    - name: Header
      args:
          header: X-Request-Id
          regexp: \d+
```

- **Method Route Predicate Factory:**
```yml
predicates:
    - Method=GET,POST
```
Or
```yml
predicates:
    name: Method
    args:
        methods: GET,POST
```

- **Path Route Predicate Factory:** takes two parameters: a list of Spring `PathMatcher` patterns and an optional flag called `matchTrailingSlash` (defaults to true).
```yml
predicates:
    - Path=/red/{segment},/blue/{segment}
```
This route matches if the request path was, for example: `/red/1` or `/red/1/` or `/red/blue` or `/blue/green`. If `matchTrailingSlash` is set to false, then request path `/red/1/` will not be matched.  

This predicate extracts the segment portion of the path and places in the request attributes object:

In [ ]:
// All RoutePathFactory classes that want to store path params use the below method
public static void putUriTemplateVariables(ServerWebExchange exchange, Map<String, String> uriVariables) {
    if (exchange.getAttributes().containsKey(URI_TEMPLATE_VARIABLES_ATTRIBUTE)) {
        Map<String, Object> existingVariables = (Map<String, Object>) exchange.getAttributes()
                .get(URI_TEMPLATE_VARIABLES_ATTRIBUTE);
        HashMap<String, Object> newVariables = new HashMap<>();
        newVariables.putAll(existingVariables);
        newVariables.putAll(uriVariables);
        exchange.getAttributes().put(URI_TEMPLATE_VARIABLES_ATTRIBUTE, newVariables);
    }
    else {
        exchange.getAttributes().put(URI_TEMPLATE_VARIABLES_ATTRIBUTE, uriVariables);
    }
}

To get the information saved:

In [ ]:
Map<String, String> uriVariables = ServerWebExchangeUtils.getUriTemplateVariables(serverExchange);
String segment = uriVariables.get("segment");

- **Weight Route Predicate Factory:** takes two arguments: group and weight (an int). The weights are calculated per group.
```yml
routes:
  - id: weight_high
    uri: https://weighthigh.org
    predicates:
      - Weight=group1, 8
  - id: weight_low
    uri: https://weightlow.org
    predicates:
      - Weight=group1, 2
```
This route would forward ~80% of traffic to weighthigh.org and ~20% of traffic to weighlow.org  

Few things to note about the way we define predicates in YAML:
- It takes the form of `Predicate=[param[,param]+]`
- Predicate above is the predicate's name, which is derived automatically from the factory class name by removing the `RoutePredicateFactory` suffix

### Custom Route Predicate
To create a custom route predicate:
1. Define a `Config` class to hold configuration parameters specific to the predicate
2. Extend `AbstractRoutePredicateFactory`, using the configuration class as its template parameter
3. Override the `apply` method, returning a `Predicate` that implements the desired test logic
4. Register `RoutePredicateFactory` with the application context

In [ ]:
public class AcceptRoutePredicateFactory extends AbstractRoutePredicateFactory<AcceptRoutePredicateFactory.Config> {
    public AcceptRoutePredicateFactory() {
        super(AcceptRoutePredicateFactory.Config.class);
    }

    @Override
    public List<String> shortcutFieldOrder() {
        return Collections.singletonList("types");
    }

    @Override
    public ShortcutType shortcutType() {
        return ShortcutType.GATHER_LIST;
    }

    @Override
    public Predicate<ServerWebExchange> apply(AcceptRoutePredicateFactory.Config config) {
        return new GatewayPredicate() {
            @Override
            public boolean test(ServerWebExchange serverWebExchange) {
                String accept = serverWebExchange.getRequest().getHeaders().getFirst(HttpHeaders.ACCEPT);
                if (StringUtils.isNotBlank(accept)) {
                    return config.getTypes().stream().anyMatch(t -> t.equalsIgnoreCase(accept));
                }

                return false;
            }
        };
    }

    @Validated
    public static class Config {
        private List<String> types = new ArrayList<>();

        public List<String> getTypes() {
            return types;
        }

        public Config setTypes(List<String> types) {
            this.types = types;
            return this;
        }
    }
}

@Configuration
public class CustomPredicatesConfig {
    @Bean
    public AcceptRoutePredicateFactory acceptTypes() {
        return new AcceptRoutePredicateFactory();
    }
}

Then in application.yaml, specify the new predicate:
```yaml
predicates:
    - Accept=application/json
```

### Route Filters
Similar to `RoutePredicateFilter`, `GatewayFilterFactory`. Spring Cloud Gateway comes with multiple filters, some of the most important ones are:

- **AddRequestHeader GatewayFilter Factory:** adds a header to downstream request’s headers for all matching requests.
```yaml
filters:
    - AddRequestHeader=X-Request-color, blue
```
`AddRequestHeader` is aware of the URI variables used to match a path or host. URI variables may be used in the value and are expanded at runtime:
```yaml
filters:
    - AddRequestHeader=X-Request-color, {color}
```

- **AddRequestParameter GatewayFilter Factory:** adds query param
```yaml
filters:
    - AddRequestParameter=color, red
```

- **RequestSize GatewayFilter Factory:** restricts a request from reaching the downstream service if request size is greater than the set limit. If the request size is greater, status 413 response is returned.
```yaml
filters:
    - name: RequestSize
      args:
        maxSize: 5000000
```
The default size unit is bytes, we can append KB, MB, etc to the number.

- **PrefixPath GatewayFilter Factory:** adds a prefix to the path:
```yaml
filters:
    - PrefixPath=/mypath
```
This will add `/mypath` prefix, so a request to `/home` would be routed to `/mypath/home`.

- **RedirectTo GatewayFilter Factory:** redirects request to given URL:
```yaml
filters:
    - RedirectTo=302, https://acme.org
```
This will make a status 302 redirect to acme.org

- **RewritePath GatewayFilter Factory:** takes a path regexp parameter and a replacement parameter. This uses Java regular expressions for a flexible way to rewrite the request path:
```yaml
filters:
    - RewritePath=/red/?(?<segment>.*), /$\{segment}
```
This would replace path `/color/blue` with `/blue`

- **SecureHeaders GatewayFilter Factory:** adds a number of security related headers as per [this](https://blog.appcanary.com/2017/http-security-headers.html).

There are a number of other filters related to circuit breaker, retrying and auth not covered here.

### Global Filters
These are special filters that are conditionally applied to all routes.  

*Gateway Web Handler* combines global filters with the selected list of route filters to create a combined filter chain. This combined filter chain is sorted by the `org.springframework.core.Ordered interface`, which we can set by implementing the `getOrder()` method. As Spring Cloud Gateway distinguishes between “pre” and “post” phases for filter logic execution, the filter with the highest precedence is the first in the “pre”-phase and the last in the “post”-phase.

- **Gateway Metrics Filter:** to enable metric collection, we need to have `spring-boot-starter-actuator` in classpath and `spring.cloud.gateway.metrics.enabled` property is not set to false.

- **ReactiveLoadBalancerClientFilter:**  looks for a URI in the exchange attribute named `ServerWebExchangeUtils.GATEWAY_REQUEST_URL_ATTR`. If the URL has a lb scheme (such as `lb://myservice`), it uses the Spring Cloud ReactorLoadBalancer to resolve the name (myservice in this example) to an actual host and port and replaces the URI in the same attribute. The unmodified original URL is appended to the list in the `ServerWebExchangeUtils.GATEWAY_ORIGINAL_REQUEST_URL_ATTR` attribute. 
```yaml
routes:
    - id: myRoute
      uri: lb://service
      predicates:
        - Path=/service/**
```
By default, when a service instance cannot be found by the `ReactorLoadBalancer`, a 503 is returned.

### Custom Gateway Filter
To write our own custom gateway filter, we need to extends `AbstractGatewayFilterFactory`:

In [ ]:
@Component
public class LoggingGatewayFilterFactory extends AbstractGatewayFilterFactory<LoggingGatewayFilterFactory.Config> {

    final Logger logger = LoggerFactory.getLogger(LoggingGatewayFilterFactory.class);

    public LoggingGatewayFilterFactory() {
        super(Config.class);
    }
    
    @Override
    public List<String> shortcutFieldOrder() {
        return Arrays.asList("baseMessage", "preLogger", "postLogger");
    }

    @Override
    public GatewayFilter apply(Config config) {
        return (exchange, chain) -> {
            // Pre-processing
            if(config.isPreLogger()) {
                logger.info("Pre GatewayFilter logging: " + config.getBaseMessage());
            }
            
            return chain.filter(exchange)
                .then(Mono.fromRunnable(() -> {
                    // Post-processing
                    if (config.isPostLogger()) {
                      logger.info("Post GatewayFilter logging: " + config.getBaseMessage());
                    }
                }));
        }
    }

    public static class Config {
        private String baseMessage;
        private boolean preLogger;
        private boolean postLogger;

        // contructors, getters and setters...
    }
}

To define this newly created filter in a `Route`:
```yaml
filters:
    - RewritePath=/service(?<segment>/?.*), $\{segment}
    - name: Logging
      args:
        baseMessage: My Custom Message
        preLogger: true
        postLogger: true
```
Or
```yaml
filters:
    - RewritePath=/service(?<segment>/?.*), $\{segment}
    - Logging=My Custom Message, true, true
```

Again, to define order of this filter within the chain, return `OrderedGatewayFilter` instead of `GatewayFilter`:

In [ ]:
// Order of filter set to 1
return new OrderedGatewayFilter((exchange, chain) -> {
    // Logic
}, 1);

### Custom Global Filter
To create a global filter, extend `GlobalFilter`:

In [ ]:
@Component
public class LoggingGlobalFilter implements GlobalFilter, Ordered {

    final Logger logger = LoggerFactory.getLogger(LoggingGlobalFilter.class);

    @Override
    public Mono<Void> filter(ServerWebExchange exchange, GatewayFilterChain chain) {
        logger.info("Pre Global Filter");
        return chain.filter(exchange)
          .then(Mono.fromRunnable(() -> {
              logger.info("Post Global Filter");
          }));
    }

    @Override
    public int getOrder() {
        return -1;
    }
}

### Modifying Request/Response 
Consider the scenario:
1. if the request contains `Accept-Language` header, it is acceptable and we retain it
2. else, we use the locale query parameter
3. if that's not present, use a default locale
4. finally, we want to remove the locale query param

In [ ]:
return (exchange, chain) -> {
    if(exchange.getRequest().getHeaders().getAcceptLanguage().isEmpty()) {
        String queryParamLocale = exchange.getRequest().getQueryParams().getFirst("locale");
        Locale requestLocale = Optional.ofNullable(queryParamLocale).map(l -> Locale.forLanguageTag(l)).orElse(config.getDefaultLocale());
        
        // To modify the request, we make use of its mutate method
        exchange.getRequest().mutate().headers(h -> h.setAcceptLanguageAsLocales(Collections.singletonList(requestLocale)));
    }
    
            
    // To mdify the URI
    ServerExchange modified = exchange.mutate().request(originalRequest -> {
        originalRequest.uri(    
            UriComponentsBuilder.fromUri(exchange.getRequest().getURI())
                .replaceQueryParams(new LinkedMultiValueMap<String, String>())
                .build()
                .toUri()
        )
    }).build();
    return chain.filter(modified);
}

Modifying response:

In [ ]:
(exchange, chain) -> {
    return chain.filter(exchange)
      .then(Mono.fromRunnable(() -> {
          // We don't need to create a copy of it to modify it, as with the request.
          ServerHttpResponse response = exchange.getResponse();

          Optional.ofNullable(exchange.getRequest()
            .getQueryParams()
            .getFirst("locale"))
            .ifPresent(qp -> {
                String responseContentLanguage = response.getHeaders()
                  .getContentLanguage()
                  .getLanguage();

                response.getHeaders()
                  .add("Custom-Language-Header", responseContentLanguage);
                });
        }));
}

### Route Metadata
We can specify arbitrary attributes to a `Route`:
```yaml
routes:
  - id: route_with_metadata
    uri: https://example.org
    metadata:
      singleAttribute: "Attribute Value"
      compositeAttribute:
        name: "value"
      numericalAttribute: 1
```
Metadata can then be fetched from the `Route` object using:

In [ ]:
Route route = exchange.getAttribute(GATEWAY_ROUTE_ATTR);
// get all metadata properties
route.getMetadata();
// get a single metadata property
route.getMetadata(someKey);

## HTTP Client
Spring Cloud Gateway provides a number of options to configure its HTTP client (Netty Client) which it uses to forward request to downstream services. Few of the configurations are covered here:

### TLS
To enable TLS on the gateway server, we use the usual Spring configuration:
```yaml
server:
  ssl:
    enabled: true
    key-alias: gateway
    key-store-password: pa$$w0rd
    key-store: classpath: keystore.p12
    key-store-type: PKCS12
```
For mutual TLS support, we'll need to configure keystore for the HTTP client:
```yaml
spring:
  cloud:
    gateway:
      httpclient:
        key-alias: gateway
        key-store-password: pa$$w0rd
        key-store: classpath: keystore.p12
        key-store-type: PKCS12
```

If we want the Spring Cloud Gateway HTTP client to now trust all downstream certificates:
```yaml
spring:
  cloud:
    gateway:
      httpclient:
        ssl:
          useInsecureTrustManager: true
```

This is not recommended and we should supply the client with all trusted certificates:
```yaml
spring:
  cloud:
    gateway:
      httpclient:
        ssl:
          trustedX509Certificates:
          - cert1.pem
          - cert2.pem
```
If we do not set certificates above, default truststore is used, which can be overridden by `javax.net.ssl.trustStore`. 

### Timeouts
TLS handshake timeout can be configured by setting `spring.gateway.httpclient.ssl.handshake-timeout-millis`. The connect and response timeouts can be set both at global and at route level:
```yaml
spring:
  cloud:
    gateway:
      httpclient:
        connect-timeout: 1000
        response-timeout: 5s
```
```yaml
- id: per_route_timeouts
  uri: https://example.org
  predicates:
    - name: Path
      args:
        pattern: /delay/{timeout}
  metadata:
    response-timeout: 200
    connect-timeout: 200
```

## DiscoveryClient Route Definition Locator
We can use `DiscoveryClient` to talk to service registry server like Eureka and generate route definitions automatically. For this, we must set:
```yaml
spring:
    cloud:
        gateway:
            discovery:
                locator:
                    enabled: true
                    lower-case-service-id: true
```

In the created route, Spring Cloud Gateway adds:
- a path predicate with pattern `/serviceId/**`. Here `serviceId` would be equivalent to application name
- a rewrite path filter with regex `/serviceId/?(?<remaining>.*)` and the replacement `/${remaining}`. Basically strips `/serviceId` from the path
- the destination URI set is loadbalanced version `lb://serviceId`

We can define list of predicates and filters to associate with all the generated route using the following:
```yaml
spring:
    cloud:
        gateway:
            discovery:
                locator:
                    predicates:
                        - Accept=application/json
                    routes:
                        - AddHeader=color, red
```